In [1]:
from motor.motor_asyncio import AsyncIOMotorClient
import asyncio
from pymongo import MongoClient
import pandas as pd


DB_HOST_DEV = "mongo"
DB_PASSWORD_DEV = "example"
DB_PORT_DEV = "27017"
DB_DATABASE_DEV = "customer-care-db"

MONGODB_HOST = "127.0.0.1"  # Ou o endereço IP/nome do servidor MongoDB
MONGODB_PORT = 27017  # Ou a porta correta em que o MongoDB está configurado para ouvir

In [7]:
DB_URL = f"""mongodb://{MONGODB_HOST}:{DB_PASSWORD_DEV}
@{DB_HOST_DEV}:{DB_PORT_DEV}/"""

# client = AsyncIOMotorClient(DB_URL)

client = MongoClient("mongodb://localhost:27017/")

database = client[DB_DATABASE_DEV]
conversations_collection = database["messages"]

script_collection = database["script"]

In [12]:
script_collection.drop()

In [ ]:
def get_descriptions_dataframe(results):
    descriptions = []
    descriptions = [i["problem"] for i in list(results)]
    return pd.DataFrame(descriptions).reset_index(drop=True)

In [ ]:
result = script_collection.find({})

In [ ]:
script_collection.find_one({"problem.id": 4})

In [ ]:
data = get_descriptions_dataframe(result)
data

In [ ]:
data = {}
data["message"] = "Makson Vinicio Ferreira de Sousa"

In [ ]:
data.get("message", "").strip()